In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

In [146]:
from scipy.special import softmax
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

class SchoolManifold():

    def __init__(self, embeddings_df: pd.DataFrame, k = 15):
        self.embeddings_df = embeddings_df
        self.embeddings = embeddings_df.iloc[:, 1:]
        self.k = k

    
    def _compute_neighbors(self, k: int):
        knn = NearestNeighbors(n_neighbors = k, algorithm="auto")
        knn.fit(self.embeddings.values)
        dists, indices = knn.kneighbors(self.embeddings.values)
        self.neighbor_dists = dists
        self.neighbor_indices = indices


    def preprocess_for_target(self, target_df: pd.DataFrame, target_col_ind: int = None, target_col_lab = None):
        if (target_col_ind is not None):
            target_col_lab = target_df.columns[target_col_ind]

        if (target_col_lab is None):
            raise ValueError("Either target_col_ind or target_col_lab must not be None")

        merged = pd.merge(self.embeddings_df, target_df, how= "inner", on = "unitid")
        
        self.embeddings = merged[self.embeddings.columns]
        self._compute_neighbors(self.k)

        self.target_df = merged[target_df.columns]
        self.features = self.target_df.drop(labels = ["unitid", target_col_lab], axis = 1)
        orig_columns = self.features.columns
        self.target = self.target_df[target_col_lab]
        self.schools = self.target_df["unitid"]

        inds_of_target_in_original = merged.index.values
        neighbor_inds_of_target_in_original = self.neighbor_indices[inds_of_target_in_original]
        neighbor_dists_of_target_col = self.neighbor_dists[inds_of_target_in_original]

        vals_of_target_col = merged[target_col_lab].values
        neighbor_vals_of_target_col =  vals_of_target_col[neighbor_inds_of_target_in_original]
        
        features_of_target_col = merged[orig_columns].values
        neighbor_features_of_target_col = features_of_target_col[neighbor_inds_of_target_in_original]

        mask = neighbor_vals_of_target_col > vals_of_target_col.reshape(-1, 1)
        neighbor_weights_of_target_col = np.where(mask, neighbor_dists_of_target_col, np.PINF)
        neighbor_weights_of_target_col = softmax(-neighbor_weights_of_target_col, axis = 1)

        self.mask = mask
        self.v_star = (neighbor_vals_of_target_col * neighbor_weights_of_target_col).sum(axis = 1)
        self.weights =  neighbor_weights_of_target_col
        self.neighbor_features = neighbor_features_of_target_col
        self.neighbor_target = neighbor_vals_of_target_col
        
        
        self.v_star = np.where(np.isnan(self.v_star), self.target.values, self.v_star)
    

    def compute_deltas(self, reg, mode = "projection"):
        self.reg = reg
        self.adjusted_features = np.sum(self.neighbor_features * self.weights[:, :, None], axis = 1)
        self.adjusted_features = np.where(np.isnan(self.adjusted_features), self.features.values, self.adjusted_features)
        self.deltas = self.adjusted_features - self.features.values

        self.adjusted_features_df = pd.DataFrame(self.adjusted_features, columns = self.features.columns)




### Yield Efficiency

In [181]:
ic_mca = pd.read_csv("../datasets/Clusters/IC_CLUSTERS_AND_MCA.csv")
ic_mca = ic_mca.loc[:, ic_mca.columns != "ic_cluster"]

yield_efficiency = pd.read_csv("../datasets/Target Features/autofeature_per_dollar_expense.csv", index_col=0)

In [184]:
yield_efficiency.drop("Total expenses", axis = 1, inplace= True)

In [185]:
manifold = SchoolManifold(ic_mca)

In [186]:
manifold.preprocess_for_target(yield_efficiency, target_col_lab="log_yield_per_dollar")

/home/acowlagi/.conda/envs/datathon/lib/python3.9/site-packages/scipy/special/_logsumexp.py:214: RuntimeWarning: invalid value encountered in subtract
  return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


### Testing Area

In [23]:
manifold = SchoolManifold(ic_mca)

In [76]:
manifold.neighbor_indices[[2,3]]

array([[6304, 6035, 5222, 6275, 5733, 5732, 6727, 5762, 6573, 6117],
       [1133, 1782, 1345, 3417, 1978, 1757, 3500, 1455,  543,  763]])

In [86]:
test = np.array([[1,2, 3], [2,3, 4], [3,4, 5], [4, 5, 6]])
mask = test > np.array([2, 3, 4, 5]).reshape(-1, 1)

altered_test = np.where(mask, test, np.PINF)
softmax(-altered_test, axis = 1)

array([[-inf, -inf,  -3.],
       [-inf, -inf,  -4.],
       [-inf, -inf,  -5.],
       [-inf, -inf,  -6.]])

In [87]:
df = pd.DataFrame(np.random.randn(50, 1), columns=list('A'))
df["E"] = 2*np.arange(0, 50)
df2 = pd.DataFrame(np.random.randn(100, 1), columns=list('B'))
df2 = pd.DataFrame(np.random.randn(100, 1), columns=list('B'))

In [74]:
df3 = pd.merge(df2.reset_index(), df,  how="right", on = "E").set_index('index')
df3

In [7]:
neighbor_values = np.arange(24).reshape(4, 3, 2)
weights = np.array([[0.3, 0.3, 0.4], [0.9, 0.1, 0], [0.6, 0.2, 0.2], [0, 0, 1]])

In [13]:
np.sum(neighbor_values * weights[:, :, None], axis = 1)

array([[ 2.2,  3.2],
       [ 6.2,  7.2],
       [13.2, 14.2],
       [22. , 23. ]])

In [12]:
neighbor_values

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5]],

       [[ 6,  7],
        [ 8,  9],
        [10, 11]],

       [[12, 13],
        [14, 15],
        [16, 17]],

       [[18, 19],
        [20, 21],
        [22, 23]]])